In [1]:
"""
Created on Sun Feb 14 21:01:54 2016

@author: Walter Martins-Filho
e-mail: walter at on.br
        waltersmartinsf at gmail.com
"""
#******************************************************************************
#Main Goal: include the time_info in the header of the images.
#******************************************************************************

print '\n Including time information in the header \n'
print '\n Loading packages .... \n'

from pyraf import iraf #IRAF
from login import * #IRAF configurations
from astropy.io import fits #import and export fits images
import glob #package for list files
import os #package for control bash commands
import useful_functions as use #useful functions
import yaml #input data without any trouble
from string import split #use to unconcanated a string in parts
from pandas import DataFrame, read_csv #work with dataframes
from photutils import CircularAperture, aperture_photometry,CircularAnnulus,Background #Photometry routines 
from astropy.table import hstack
from astropy.io.misc import fnpickle, fnunpickle
import numpy as np
from scipy import array
print '\n.... Done.'


 Including time information in the header 


 Loading packages .... 



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |    
      |                   STSDAS Version 3.17                      |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+
stsdas/:
 analysis/      examples        hst_calib/      sobsolete/
 contrib/       fitsio/         playpen/        toolbox/
 describe       graphics/       problem

In [2]:
import time

In [3]:
%matplotlib inline
import pylab

import matplotlib.pyplot as plt #plot library
def init_plotting():
    """
    Funcao para definir uma janela grafica com parametros customizados para aptresentacoes em ppt ou latex.
    """
    plt.rcParams['figure.figsize'] = (14.0,8.0)
    plt.rcParams['font.size'] = 20
    #plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['axes.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['axes.titlesize'] = 0.75*plt.rcParams['font.size']
    plt.rcParams['legend.fontsize'] = 0.65*plt.rcParams['font.size']
    plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['xtick.major.size'] = 3
    plt.rcParams['xtick.minor.size'] = 3
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['xtick.minor.width'] = 1
    plt.rcParams['ytick.major.size'] = 3
    plt.rcParams['ytick.minor.size'] = 3
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['ytick.minor.width'] = 1
    plt.rcParams['legend.frameon'] = True
    plt.rcParams['legend.loc'] = 'best'
    plt.rcParams['axes.linewidth'] = 1

init_plotting()

In [4]:
cd /home/walter/github/iraf_task/

/home/walter/github/iraf_task


In [5]:
#input data
print '\nReading input*.yaml file ....\n'
#******************************************************************************
#**************** BEGIN INPUT PATH FILE ***************************************
#******************************************************************************
#path for your data directory, path for your data save, and names for the lists
#Import with yaml file: input path and prefix information for files
input_file = glob.glob('input*.yaml')
if input_file:
    if len(input_file) == 1:
        print 'reading input file ... \n'
        file = yaml.load(open(input_file[0])) #creating our dictionary of input variables
        data_path = file['data_path']
        save_path = file['save_path']
        planet = file['exoplanet']
        print '....  done! \n'
    if len(input_file) > 1:
        print 'reading input file ... \n'
        print '.... there is more than 1 input_path*.yaml.\n \nPlease, remove the others files that you do not need. \n'
        raise SystemExit
else:
    print 'There is no input_path*.yaml. \nPlease, create a input file describe in INPUT_PARAMETERS.'
    raise SystemExit
#******************************************************************************
#******************* END INPUT PATH FILE **************************************
#******************************************************************************


print 'YAML information:'
input_file = file
print input_file


Reading input*.yaml file ....

reading input file ... 

....  done! 

YAML information:
{'u.DEC': 'degree', 'frame': 'fk5', 'data_path': '/home/walter/exoplanet/data/xo2b/xo2b.b', 'lat-obs': 32.4164722222, 'observatory': 'mtbigelow', 'circular_orbit': False, 'gain': 3.19, 'skysection': [526, 81], 'lon-obs': -110.734527778, 'DEC': '+50:13:32.9', 'apertures': [7, 14], 'pxpositions': [582.79, 581.76, 559.06, 512.4], 'u.RA': 'degree', 'altitude': 2510.0, 'time-obs': 'UT', 'time_zone': 7.0, 'date-obs': 'DATE-OBS', 'epoch': '2000.0', 'RA': '07:48:06.46', 'save_path': '/home/walter/exoplanet/data/xo2b/xo2b.b/iraf_task', 'time': 'ut', 'exoplanet': 'xo2b', 'scale-time': 'utc'}


In [6]:
os.chdir(save_path)

In [7]:
ls

ABxo2b.0002.fits  ABxo2b.0096.fits  ABxo2b.0190.fits
ABxo2b.0004.fits  ABxo2b.0098.fits  ABxo2b.0192.fits
ABxo2b.0006.fits  ABxo2b.0100.fits  ABxo2b.0194.fits
ABxo2b.0008.fits  ABxo2b.0102.fits  ABxo2b.0196.fits
ABxo2b.0010.fits  ABxo2b.0104.fits  ABxo2b.0198.fits
ABxo2b.0012.fits  ABxo2b.0106.fits  ABxo2b.0200.fits
ABxo2b.0014.fits  ABxo2b.0108.fits  ABxo2b.0202.fits
ABxo2b.0016.fits  ABxo2b.0110.fits  ABxo2b.0204.fits
ABxo2b.0018.fits  ABxo2b.0112.fits  ABxo2b.0206.fits
ABxo2b.0020.fits  ABxo2b.0114.fits  ABxo2b.0208.fits
ABxo2b.0022.fits  ABxo2b.0116.fits  ABxo2b.0210.fits
ABxo2b.0024.fits  ABxo2b.0118.fits  ABxo2b.0212.fits
ABxo2b.0026.fits  ABxo2b.0120.fits  ABxo2b.0214.fits
ABxo2b.0028.fits  ABxo2b.0122.fits  ABxo2b.0216.fits
ABxo2b.0030.fits  ABxo2b.0124.fits  ABxo2b.0218.fits
ABxo2b.0032.fits  ABxo2b.0126.fits  ABxo2b.0220.fits
ABxo2b.0034.fits  ABxo2b.0128.fits  ABxo2b.0222.fits
ABxo2b.0036.fits  ABxo2b.0130.fits  ABxo2b.0224.fits
ABxo2b.0038.fits  ABxo2b.0132.fits  ABxo2b.022

In [8]:
def bkg_info(data_path,save_path,input_file):
    """
    Obtain the sky backgound for each science image.
    
    More in: http://photutils.readthedocs.io/en/latest/api/photutils.background.Background.html#photutils.background.Background
    
    WARNING:
    
    This routine only need to be run one time for the same set of images.
    ___
    

    INPUT:
    For obtain this parameters, use the input_info function.

    data_path: string, path where are the images data.
    save_path: string, path where will save all reduced images.
    input_file: dict, with information describe in the YAML file.
    """
    #set the original directory
    original_path = os.getcwd()
    os.chdir(save_path)
    planet = input_file['exoplanet']
    tempo = time.time()
    print 'Obtain background data for each image ... \n'
    if not os.path.exists('background'): #if background does not exist, create!
        os.makedirs('background')
    
    images = sorted(glob.glob('AB'+planet+'*.fits'))
    #if background exist, check if files bkg_data_image_name_.pik exist
    #if not exist, then create, else: get out of here! XD
    if os.path.exists('background') == True : 
        value = [] 
        for i in images:
            value.append(os.path.isfile('./background/'+'bkg_data_'+i+'_.pik'))
    if (False in value) == True:
        print 'Does not exist all files to all images in the sample.'
        print 'Calculating ...'
        print 'This will take some time... go drink some coffe'
        print ' while you wait for the routine finish \n'
        for i in images:
            im = fits.getdata(i,header=False)
            im = np.array(im,dtype='Float64')
            bkg = Background(im,tuple(input_file['skysection'])) #estimating the background using a boxpixel
            fnpickle(bkg,'./background/'+'bkg_data_'+i+'_.pik')
            use.update_progress((float(images.index(i))+1.)/len(images))
    else:
        print 'The sky background files *.pik exist. \n'
    print 'Sky backgound obtained.'
    print 'Total time = ',abs(time.time()-tempo)/60.,' minutes'
    os.chdir(original_path)
    return 

In [9]:
bkg_info(data_path,save_path,input_file)

Obtain background data for each image ... 

The sky background files *.pik exist. 

Sky backgound obtained.
Total time =  0.000246600310008  minutes


In [10]:
# from numba import jit

In [11]:
def bkg_read(data_path,save_path, input_file):
    """
    Read the sky backgound files obtained by bkg_info routine and return the background mask and noise data.
    """
    #set the original directory
    tempo = time.time()
    original_path = os.getcwd()
    planet = input_file['exoplanet']
    #change to save data reduction directory
    os.chdir(save_path)
    print 'Reading sky backgound files ...'
    images = sorted(glob.glob('AB'+planet+'*.fits'))
    bkg_data = []
    bkg_rms = []
    for i in range(len(images)):
        bkg = fnunpickle('./background/'+'bkg_data_'+images[i]+'_.pik')
        bkg_data.append(bkg.background)
        bkg_rms.append(bkg.background_rms)
        use.update_progress((i+1.)/len(images))
    os.chdir(original_path)
    print 'total time = ',abs(time.time()-tempo)/60.
    return bkg_data, bkg_rms

In [12]:
bkg_data, bkg_rms = bkg_read(data_path,save_path, input_file)

Reading sky backgound files ...
Percent: [##########] 100% Done...
total time =  3.78022879759


In [13]:
def phot_aperture(data_path,save_path,input_file,bkg_data,bkg_rms):
    """
    Obtain the aperture photometry to the list of apertures in the input_file dictionary.
    ___
    INPUT:
    For obtain this parameters, use the input_info function.

    data_path: string, path where are the images data.
    save_path: string, path where will save all reduced images.
    input_file: dict, with information describe in the YAML file.
    bkg_data: sky background mask from bkg_info routine
    bkg_rms: sky background noise from bkg_info routine

    """
    #set the original directory
    original_path = os.getcwd()
    planet = input_file['exoplanet']
    radii = np.arange(input_file['apertures'][0],input_file['apertures'][1],0.1)
    #change to save data reduction directory
    os.chdir(save_path)
    if not os.path.exists('phot_results'):
        os.makedirs('phot_results')
    tempo = time.time()
    print 'Starting aperture photometry'
    print 'Saving results on: '+save_path+'/phot_results/'
    #check the number of objects to make the photometry
    N_obj = len(input_file['pxpositions'])/2.
    print 'Number of objects = ',N_obj
    positions = [] #create the positions variable (X,Y) in pixels unit on the CCD
    for i in range(len(input_file['pxpositions'])):
        if i % 2 == 0: #if the number is a even (or not a odd), the turple is created
            positions.append((input_file['pxpositions'][i],input_file['pxpositions'][i+1]))
    print 'Radius from ',radii[0],' to ',radii[-1],'\n'
    images = sorted(glob.glob('AB'+planet+'*.fits'))
    for radius in radii:
        flux_data = []
        for i in range(len(images)):
            im = fits.getdata(images[i],header=False)
            im = array(im,dtype='Float64')
            phot_table = aperture_photometry(im - bkg_data[i], CircularAperture(positions, radius),
                                             error=bkg_rms[i], effective_gain=float(input_file['gain']))
            flux = [phot_table['aperture_sum'][0], phot_table['aperture_sum'][1],phot_table['aperture_sum_err'][0],
                    phot_table['aperture_sum_err'][1],images[i]]
            flux_data.append(flux)
        flux_data = DataFrame(flux_data,columns=['hoststar','refstar','hoststar_err','refstar_err','image'])
        flux_data.to_csv('./phot_results/'+planet+'_flux_radius_'+str(radius)+'.csv')
        use.update_progress((float(np.where(radii == radius)[0])+1.)/len(radii))
    print 'Time total = ',abs(time.time()-tempo)/60.,' minutes'
    os.chdir(original_path)

In [14]:
phot_aperture(data_path,save_path,input_file,bkg_data,bkg_rms)

Starting aperture photometry
Saving results on: /home/walter/exoplanet/data/xo2b/xo2b.b/iraf_task/phot_results/
Number of objects =  2.0
Radius from  7.0  to  13.9 

Percent: [##########] 100% Done...
Time total =  3.45870695114  minutes
